In [3]:
%run "../00_project_config.ipynb"

import pathlib as pl
import pandas as pd

annotated_files_table = pl.Path(
    "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/annotations/autogen",
    "annotated_files.tsv"
)

annotated_files = pd.read_csv(annotated_files_table, header=0, sep="\t", comment="#")

read_stats_file = pl.Path(
    "/home/ebertp/work/projects/hgsvc/2023_batch_data_tables",
    "hgsvc3_batch1_read_stats.tsv"
)

read_stats = pd.read_csv(read_stats_file, header=0, sep="\t")

if "batch1" in read_stats_file.name:
    out_batch_summary_table = PROJECT_BASE.joinpath(
        "annotations", "data_freezes", "HGSVC3_data_batch1_file_details.tsv"
    )
    add_batch_column = 1
elif "batch2" in read_stats_file.name:
    out_batch_summary_table = PROJECT_BASE.joinpath(
        "annotations", "data_freezes", "HGSVC3_data_batch2_file_details.tsv"
    )
    add_batch_column = 2
else:
    raise

file_sources = []
for row in read_stats.itertuples(index=True):
    if row.source == "all":
        file_sources.append("full_dataset")
        continue
    short_source = row.source.split(".")[0]
    matched = annotated_files.loc[annotated_files["file_name"].str.contains(short_source), :]
    if matched.empty:
        raise ValueError(short_source)
    if matched.shape[0] > 1:
        raise ValueError(f"{short_source} - {matched}")
    file_path = matched["file_rel_path"].values[0]
    file_path = file_path.split("/", 3)[-1]
    file_sources.append(file_path)
    
file_sources = pd.DataFrame(file_sources, index=read_stats.index, columns=["rel_path"])

read_stats = pd.concat([read_stats, file_sources], axis=1, ignore_index=False)
read_stats["batch_number"] = add_batch_column

samples_in_batch = read_stats["sample"].nunique()

with open(out_batch_summary_table, "w") as dump:
    _ = dump.write("# AUTOGEN TABLE - DO NOT EDIT\n")
    _ = dump.write(f"# {TODAY}\n")
    _ = dump.write(f"# Samples in batch: {samples_in_batch}\n")
    read_stats.to_csv(dump, sep="\t", header=True, index=False)

print(f"Samples in batch number {add_batch_column}:")
print(samples_in_batch)

Samples in batch number 1:
39
